In [1]:
import os
import torch
from dataloader import MRDataset
from tqdm import tqdm
from tqdm import tqdm_notebook


In [2]:
def extract_predictions(task, plane, train=True):
    assert task in ['acl', 'meniscus', 'abnormal']
    assert plane in ['axial', 'coronal', 'sagittal']
    
    models = os.listdir('/home/niamh/Documents/MRNET/models/')

    model_name = list(filter(lambda name: task in name and plane in name, models))[0]
    model_path = f'/home/niamh/Documents/MRNET/models/{model_name}'

    mrnet = torch.load(model_path)
    _ = mrnet.eval()
    
    train_dataset = MRDataset('/home/niamh/Documents/MRNET/data/', 
                              task, 
                              plane, 
                              transform=None, 
                              train=train, 
                             # normalize=False
                             )
    
    train_loader = torch.utils.data.DataLoader(train_dataset, 
                                               batch_size=1, 
                                               shuffle=False, 
                                               num_workers=10, 
                                               drop_last=False)
    predictions = []
    labels = []
    with torch.no_grad():
        for image, label, _ in tqdm_notebook(train_loader):
            logit = mrnet(image)
            prediction = torch.sigmoid(logit)
            predictions.append(prediction[0][1].item())
            labels.append(label.item())

    return predictions, labels

In [ ]:
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

task = 'acl'
results = {}

for plane in ['axial', 'coronal', 'sagittal']:
    predictions, labels = extract_predictions(task, plane)
    results['labels'] = labels
    results[plane] = predictions
    
X = np.zeros((len(predictions), 3))
X[:, 0] = results['axial']
X[:, 1] = results['coronal']
X[:, 2] = results['sagittal']

y = np.array(labels)

logreg = LogisticRegression(solver='lbfgs')
logreg.fit(X, y)

<ipython-input-2-12d1eff12572>:29: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for image, label, _ in tqdm_notebook(train_loader):


In [ ]:
y_pred = logreg.predict_proba(X)[:, 1]
metrics.roc_auc_score(y, y_pred)

In [110]:
final = pd.concat([pd.DataFrame(X), pd.DataFrame(y), pd.DataFrame(y_pred)], axis =1)
final.columns = ['axial','coronal', 'sagittal', 'label','prediction']
final.head()

,axial,coronal,sagittal,label,prediction
0,0.416907,0.235121,0.057556,0.0,0.060770
1,0.988462,0.848889,0.882496,1.0,0.910774
2,0.070707,0.239407,0.162977,0.0,0.011111
3,0.173366,0.416141,0.257963,0.0,0.031665
4,0.113855,0.027642,0.076237,0.0,0.007486


In [111]:
pd.DataFrame(final).to_csv('acl_predictions_training')

In [80]:
task = 'acl'
results_val = {}

for plane in ['axial', 'coronal', 'sagittal']:
    predictions, labels = extract_predictions(task, plane, train=False)
    results_val['labels'] = labels
    results_val[plane] = predictions

X_val = np.zeros((len(predictions), 3))
X_val[:, 0] = results_val['axial']
X_val[:, 1] = results_val['coronal']
X_val[:, 2] = results_val['sagittal']

y_val = np.array(labels)

y_pred = logreg.predict_proba(X_val)[:, 1]
metrics.roc_auc_score(y_val, y_pred)

<ipython-input-74-12d1eff12572>:29: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for image, label, _ in tqdm_notebook(train_loader):


0.9604377104377104

In [88]:
pd.DataFrame(X_val).to_csv('acl_predictions_validation')

In [91]:
task = 'abnormal'
results = {}

for plane in ['axial', 'coronal', 'sagittal']:
    predictions, labels = extract_predictions(task, plane)
    results['labels'] = labels
    results[plane] = predictions
    
X = np.zeros((len(predictions), 3))
X[:, 0] = results['axial']
X[:, 1] = results['coronal']
X[:, 2] = results['sagittal']

y = np.array(labels)

logreg = LogisticRegression(solver='lbfgs')
logreg.fit(X, y)

<ipython-input-74-12d1eff12572>:29: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for image, label, _ in tqdm_notebook(train_loader):


LogisticRegression()

In [92]:
pd.DataFrame(X).to_csv('abnormal_predictions_training')

In [93]:
task = 'abnormal'
results_val = {}

for plane in ['axial', 'coronal', 'sagittal']:
    predictions, labels = extract_predictions(task, plane, train=False)
    results_val['labels'] = labels
    results_val[plane] = predictions

X_val = np.zeros((len(predictions), 3))
X_val[:, 0] = results_val['axial']
X_val[:, 1] = results_val['coronal']
X_val[:, 2] = results_val['sagittal']

y_val = np.array(labels)

y_pred = logreg.predict_proba(X_val)[:, 1]
metrics.roc_auc_score(y_val, y_pred)

<ipython-input-74-12d1eff12572>:29: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for image, label, _ in tqdm_notebook(train_loader):


0.9132631578947368

In [94]:
pd.DataFrame(X_val).to_csv('abnormal_predictions_validation')

In [95]:
task = 'meniscus'
results = {}

for plane in ['axial', 'coronal', 'sagittal']:
    predictions, labels = extract_predictions(task, plane)
    results['labels'] = labels
    results[plane] = predictions
    
X = np.zeros((len(predictions), 3))
X[:, 0] = results['axial']
X[:, 1] = results['coronal']
X[:, 2] = results['sagittal']

y = np.array(labels)

logreg = LogisticRegression(solver='lbfgs')
logreg.fit(X, y)

<ipython-input-74-12d1eff12572>:29: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for image, label, _ in tqdm_notebook(train_loader):


LogisticRegression()

In [96]:
pd.DataFrame(X).to_csv('meniscus_predictions_training')

In [99]:
task = 'meniscus'
results_val = {}

for plane in ['axial', 'coronal', 'sagittal']:
    predictions, labels = extract_predictions(task, plane, train=False)
    results_val['labels'] = labels
    results_val[plane] = predictions

X_val = np.zeros((len(predictions), 3))
X_val[:, 0] = results_val['axial']
X_val[:, 1] = results_val['coronal']
X_val[:, 2] = results_val['sagittal']

y_val = np.array(labels)

y_pred = logreg.predict_proba(X_val)[:, 1]
metrics.roc_auc_score(y_val, y_pred)

<ipython-input-74-12d1eff12572>:29: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for image, label, _ in tqdm_notebook(train_loader):


0.8201357466063349

In [100]:
pd.DataFrame(X_val).to_csv('meniscus_predictions_validation')

In [101]:
(0.96+0.91+0.82)/3

0.8966666666666666

In [82]:
y_pred = logreg.predict_proba(X_val)[:, 0]
metrics.roc_auc_score(y_val, y_pred)

0.039562289562289556

In [83]:
y_pred = logreg.predict_proba(X_val)[:, 2]
metrics.roc_auc_score(y_val, y_pred)

IndexError: index 2 is out of bounds for axis 1 with size 2

In [63]:
X_val = np.zeros((len(predictions), 3))
X_val[:, 0] = results_val['axial']
y_val = np.array(labels)

y_pred = logreg.predict_proba(X_val)[:, 1]
metrics.roc_auc_score(y_val, y_pred)

0.9573512906846241

In [71]:
len(results['axial'])

1130

In [ ]:
task = 'acl'
results_val = {}

for plane in ['axial', 'coronal', 'sagittal']:
    predictions, labels = extract_predictions(task, plane, train=False)
    results_val['labels'] = labels
    results_val[plane] = predictions

y_pred = logreg.predict_proba(X_val)[:, 1]
metrics.roc_auc_score(y_val, y_pred)

In [ ]:
task = 'abnormal'
results = {}

for plane in ['axial', 'coronal', 'sagittal']:
    predictions, labels = extract_predictions(task, plane)
    results['labels'] = labels
    results[plane] = predictions
    
X = np.zeros((len(predictions), 3))
X[:, 0] = results['axial']
X[:, 1] = results['coronal']
X[:, 2] = results['sagittal']

y = np.array(labels)

logreg = LogisticRegression(solver='lbfgs')
logreg.fit(X, y)

In [ ]:
task = 'abnormal'
results_val = {}

for plane in ['axial', 'coronal', 'sagittal']:
    predictions, labels = extract_predictions(task, plane, train=False)
    results_val['labels'] = labels
    results_val[plane] = predictions

y_pred = logreg.predict_proba(X_val)[:, 1]
metrics.roc_auc_score(y_val, y_pred)

In [ ]:
task = 'meniscus'
results = {}

for plane in ['axial', 'coronal', 'sagittal']:
    predictions, labels = extract_predictions(task, plane)
    results['labels'] = labels
    results[plane] = predictions
    
X = np.zeros((len(predictions), 3))
X[:, 0] = results['axial']
X[:, 1] = results['coronal']
X[:, 2] = results['sagittal']

y = np.array(labels)

logreg = LogisticRegression(solver='lbfgs')
logreg.fit(X, y)

In [ ]:
task = 'meniscus'
results_val = {}

for plane in ['axial', 'coronal', 'sagittal']:
    predictions, labels = extract_predictions(task, plane, train=False)
    results_val['labels'] = labels
    results_val[plane] = predictions

y_pred = logreg.predict_proba(X_val)[:, 1]
metrics.roc_auc_score(y_val, y_pred)

In [ ]:
import cv2
img = cv2.imread('./data/Elephant/data/05fig34.jpg')
heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
heatmap = np.uint8(255 * heatmap)
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
superimposed_img = heatmap * 0.4 + img
cv2.imwrite('./map.jpg', superimposed_img)